# Prompt Generation
## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:

from utils import  AZURE_OPENAI_API_GPT_4o_MODEL, client
model = AZURE_OPENAI_API_GPT_4o_MODEL

## Prompt generation 
A meta-prompt instructs the model to create a good prompt based on your task description or improve an existing one. 

In [9]:

META_PROMPT = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
""".strip()


def generate_prompt(task_or_prompt: str):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": META_PROMPT,
            },
            {
                "role": "user",
                "content": "Task, Goal, or Current Prompt:\n" + task_or_prompt,
            },
        ],
    )

    return completion.choices[0].message.content


PROMT_TO_IMRPOVE = generate_prompt("summarize an article")
print(PROMT_TO_IMRPOVE)


Summarize the provided article by capturing its key points, main arguments, and essential details in a concise manner.

# Steps

1. **Comprehend the Article**: Carefully read and understand the content of the provided article, paying attention to its structure (e.g., introduction, body, conclusion). 
2. **Identify Key Elements**:
   - Extract the main topic or overarching theme of the article.
   - Pinpoint the central arguments or main ideas presented.
   - Note any significant supporting details or examples essential to understanding the article’s purpose.
3. **Eliminate Redundancy**: Remove any unnecessary repetition, lengthy quotes, or minor details that do not contribute heavily to the overall meaning or objective.
4. **Organize the Summary**:
   - Start with the main topic or theme in one or two sentences.
   - Follow with the main ideas or arguments in a logical sequence.
   - Conclude with the broader implications or conclusions, if applicable.
5. **Simplify and Clarify**: Use 

### Prompt Edit

To edit prompts, we use a slightly modified meta-prompt. While direct edits are straightforward to apply, identifying necessary changes for more open-ended revisions can be challenging. To address this, we include a reasoning section at the beginning of the response. This section helps guide the model in determining what changes are needed by evaluating the existing prompt's clarity, chain-of-thought ordering, overall structure, and specificity, among other factors. The reasoning section makes suggestions for improvements and is then parsed out from the final response.

In [11]:

META_PROMPT = """
Given a current prompt and a change description, produce a detailed system prompt to guide a language model in completing the task effectively.

Your final output will be the full corrected prompt verbatim. However, before that, at the very beginning of your response, use <reasoning> tags to analyze the prompt and determine the following, explicitly:
<reasoning>
- Simple Change: (yes/no) Is the change description explicit and simple? (If so, skip the rest of these questions.)
- Reasoning: (yes/no) Does the current prompt use reasoning, analysis, or chain of thought? 
    - Identify: (max 10 words) if so, which section(s) utilize reasoning?
    - Conclusion: (yes/no) is the chain of thought used to determine a conclusion?
    - Ordering: (before/after) is the chain of though located before or after 
- Structure: (yes/no) does the input prompt have a well defined structure
- Examples: (yes/no) does the input prompt have few-shot examples
    - Representative: (1-5) if present, how representative are the examples?
- Complexity: (1-5) how complex is the input prompt?
    - Task: (1-5) how complex is the implied task?
    - Necessity: ()
- Specificity: (1-5) how detailed and specific is the prompt? (not to be confused with length)
- Prioritization: (list) what 1-3 categories are the MOST important to address.
- Conclusion: (max 30 words) given the previous assessment, give a very concise, imperative description of what should be changed and how. this does not have to adhere strictly to only the categories listed
</reasoning>
    
# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Examples: Include high-quality examples if helpful, using placeholders [in brackets] for complex elements.
   - What kinds of examples may need to be included, how many, and whether they are complex enough to benefit from placeholders.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

[Concise instruction describing the task - this should be the first line in the prompt, no section header]

[Additional details as needed.]

[Optional sections with headings or bullet points for detailed steps.]

# Steps [optional]

[optional: a detailed breakdown of the steps necessary to accomplish the task]

# Output Format

[Specifically call out how the output should be formatted, be it response length, structure e.g. JSON, markdown, etc]

# Examples [optional]

[Optional: 1-3 well-defined examples with placeholders if necessary. Clearly mark where examples start and end, and what the input and output are. User placeholders as necessary.]
[If the examples are shorter than what a realistic example is expected to be, make a reference with () explaining how real examples should be longer / shorter / different. AND USE PLACEHOLDERS! ]

# Notes [optional]

[optional: edge cases, details, and an area to call or repeat out specific important considerations]
[NOTE: you must start with a <reasoning> section. the immediate next token you produce should be <reasoning>]
""".strip()

PROMT_TO_IMRPOVE = """
Generate a concise and coherent summary of a given article, capturing its main points, arguments, and conclusions while omitting irrelevant details.

# Steps

1. Read and analyze the article or provided content thoroughly to understand its key ideas.
2. Identify:
   - **Main topic or thesis**: What is the article primarily about?
   - **Key arguments or findings**: Highlight the most important points or evidence.
   - **Conclusion or implications**: What is the takeaway or final argument?
3. Condense the content without losing clarity or altering the original meaning, aiming for brevity.
4. Avoid including minor details, personal opinions, or assumptions not stated in the source provided.
5. Maintain neutrality and a professional tone.

# Output Format

- A single coherent paragraph summarizing the article.
- If requested, provide the summary in **bullet points** for easier reading.

# Example

**Input:**
"Article discussing the environmental impact of single-use plastics, highlighting their contribution to ocean pollution, challenges with recycling, and the need for policy changes to promote sustainable alternatives."

**Output:**
The article examines the environmental repercussions of single-use plastics, particularly their role in ocean pollution and the inefficiencies of recycling systems. It emphasizes the urgency for policy reforms to encourage the adoption of sustainable alternatives and reduce the widespread dependence on disposable plastics.

*(Note: Real examples should reflect the content provided and include placeholders where necessary.)*

# Notes

- Ensure the summary captures the tone and intent of the original article.
- Length can vary depending on the complexity of the input but should remain concise. The typical range is 50-150 words unless otherwise specified by the user.
- If no specific article is provided, clarify user requirements for the task.
""".strip()

def generate_prompt(task_or_prompt: str):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": META_PROMPT,
            },
            {
                "role": "user",
                "content": "Task, Goal, or Current Prompt:\n" + task_or_prompt,
            },
        ],
    )

    return completion.choices[0].message.content

print(generate_prompt(PROMT_TO_IMRPOVE))

<reasoning>
- Simple Change: no
- Reasoning: yes
    - Identify: Steps 1-3
    - Conclusion: yes
    - Ordering: before
- Structure: yes
- Examples: yes
    - Representative: 4
- Complexity: 3
    - Task: 3
    - Necessity: 4
- Specificity: 4
- Prioritization: reasoning clarity, structure, example usability
- Conclusion: Reframe for clarity, enhance example representativeness, and emphasize reasoning/conclusion ordering where necessary.
</reasoning>

Generate a concise and coherent summary of a given article, capturing its main points, arguments, and conclusions while omitting irrelevant details.

# Steps

1. Thoroughly read and analyze the provided article to extract key ideas.
2. Identify:
   - **Main topic or thesis**: Determine the article's primary subject or argument.
   - **Key arguments or findings**: Summarize the most important points, evidence, or insights presented.
   - **Conclusion or implications**: Note the primary takeaway, resolution, or projected impact.
3. Use the i

### Congrats!

You've learned how to generate prompts using LLM, you're ready to move to the next chapter. Happy prompting!